# Category-Location Annual Heatmap

This notebook generates and exports a heatmap of species sightings for a category for a given location and year.

In [1]:
%run ../pathutils.ipynb
%run ../definitions.ipynb
%run ../database.ipynb
%run ../export.ipynb
%run pathutils.ipynb

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

def export_heatmap_chart(export_folder_path, data, location, category, year):
    # Create a new figure
    fig = plt.figure(figsize=(12, data.shape[0] / 3))

    # Plot the heatmap
    sns.heatmap(data, cmap="YlOrRd", annot=False)

    # Set the axis labels
    plt.title(f"Number of Sightings of {category} at {location} in {year}")
    plt.xlabel("")
    plt.ylabel("")

    # Export the chart
    export_chart(export_folder_path, "heatmap", "png")

    # Close the figure
    plt.close(fig)

In [ ]:
country = None
location = None
category = None
year = None

In [ ]:
import calendar

# Load the definitions
definitions = load_definitions("wildlife", "heatmap", country, location, category, year)

# Iterate over each definition to produce a report for each one
for definition in definitions:
    for year in range(int(definition["From"]), 1 + int(definition["To"])):
        # Construct the query
        query = construct_query("wildlife", "annual_category_location_heatmap.sql", {
            "LOCATION": definition["Location"],
            "CATEGORY": definition["Category"],
            "SPECIES": "",
            "YEAR": str(year)
        })

        try:
            # Run the query to load the data - this raises a ValueError if there's no data
            df = query_data("wildlife", query)

            # Pre-process the data to provide a heatmap data source
            df["Month"] = df["Date"].dt.month
            heatmap_data = df.groupby(["Species", "Month"])["Count"].sum().unstack(fill_value=0)

            # Make sure all months are represented even if there are no sightings in the data set for that month
            for month in range(1, 13):
                if month not in heatmap_data.columns:
                    heatmap_data[month] = 0

            # Re-order the columns in calendar order
            heatmap_data = heatmap_data[sorted(heatmap_data.columns)]

            # Set the labels to the month abbreviations rather than month numbers
            heatmap_data.columns = [calendar.month_abbr[m] for m in heatmap_data.columns]

            # Export the data to Excel
            export_folder_path = get_annual_heatmap_export_folder_path(definition["Country"], definition["Location"], definition["Category"], year)
            export_to_spreadsheet(export_folder_path, "heatmap.xlsx", { "Sightings": heatmap_data })

            # Export the chart
            export_heatmap_chart(export_folder_path, heatmap_data, definition["Location"], definition["Category"], year)

        except ValueError:
            print(f"WARNING: No data found for location = {definition['Location']}, category = {definition['Category']}")
